In [ ]:
!pip install osmnx

In [1]:
import pandas as pd
import numpy as np
import osmnx as ox

In [2]:
#Define Haversine class for calculating shortest distance between accident position and "nodes" which extracted using "osmnx" library.
import math
def haversine(lon1, lat1, lon2, lat2):
    """
    Calculate the great circle distance between two points 
    on the earth (specified in decimal degrees)
    """
    # convert decimal degrees to radians 
    lon1 = math.radians(lon1)
    lat1 = math.radians(lat1)
    lon2 = np.radians(lon2)
    lat2 = np.radians(lat2)
    # lon1, lat1, lon2, lat2 = map(np.radians, [lon1, lat1, lon2, lat2])

    # haversine formula 
    dlon = lon2 - lon1 
    dlat = lat2 - lat1 
    a = np.sin(dlat/2)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon/2)**2
    c = 2 * np.arcsin(np.sqrt(a)) 
    r = 6378 # Radius of earth in kilometers
    return c * r

In [3]:
G = ox.graph_from_place('Los Angeles, California', network_type='drive')

In [4]:
!pip install pandas --upgrade

Requirement already up-to-date: pandas in /Users/lyudonghang/opt/anaconda3/envs/interface/lib/python3.7/site-packages (1.3.5)


In [4]:
# load data from kaggle accident dataset and extract data of LA with the year of 2020
data_path = "US_Accidents_Dec21_updated.csv"
data = pd.read_csv(data_path)
kaggle_LA = data[data["City"]=="Los Angeles"]
print(kaggle_LA.shape)
kaggle_LA['Start_Time'] =  pd.to_datetime(kaggle_LA['Start_Time'], format='%Y-%m-%d %H:%M:%S')
kaggle_LA['Year'] = pd.DatetimeIndex(kaggle_LA['Start_Time']).year
kaggle_LA_2020 = kaggle_LA[kaggle_LA["Year"]==2020]
print(kaggle_LA_2020.shape)

(68956, 47)
(16509, 48)


/Users/lyudonghang/opt/anaconda3/envs/interface/lib/python3.7/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/Users/lyudonghang/opt/anaconda3/envs/interface/lib/python3.7/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys


In [5]:
# add negative samples through changing coordinate a bit
def addNegativeSamples(df):
    global balanced_kaggle_california_df
    #Specifying raw dataframe
    raw_df = df.copy()
    raw_df["Accident"] = 1

    #Specifying upper coordinate dataframe
    negative_upper_coordinate_df = df.copy()    
    negative_upper_coordinate_df["Accident"] = 0

    #Specifying lower coordinate dataframe
    negative_lower_coordinate_df = df.copy()
    negative_lower_coordinate_df["Accident"] = 0
    

    #Offset "LONGITUD" and "LATITUDE" with +/- ~100 meters
    # negative_upper_coordinate_df["Start_Lng"] = negative_upper_coordinate_df["Start_Lng"].apply(lambda x: x-random.uniform((x/750), (x/300)))
    # negative_upper_coordinate_df["Start_Lat"] = negative_upper_coordinate_df["Start_Lat"].apply(lambda x: x-random.uniform((x/750), (x/300)))
    negative_upper_coordinate_df["Start_Lng"] = negative_upper_coordinate_df["Start_Lng"].apply(lambda x: x-(x/750))
    negative_upper_coordinate_df["Start_Lat"] = negative_upper_coordinate_df["Start_Lat"].apply(lambda x: x-(x/750))


    # negative_lower_coordinate_df["Start_Lng"] = negative_lower_coordinate_df["Start_Lng"].apply(lambda x: x+random.uniform((x/750), (x/300)))
    # negative_lower_coordinate_df["Start_Lat"] = negative_lower_coordinate_df["Start_Lat"].apply(lambda x: x+random.uniform((x/750), (x/300)))   
    negative_lower_coordinate_df["Start_Lng"] = negative_lower_coordinate_df["Start_Lng"].apply(lambda x: x+(x/750))
    negative_lower_coordinate_df["Start_Lat"] = negative_lower_coordinate_df["Start_Lat"].apply(lambda x: x+(x/750))   

    balanced_kaggle_california_df = pd.concat([raw_df, negative_lower_coordinate_df, negative_upper_coordinate_df],ignore_index=True)
    return balanced_kaggle_california_df

kaggle_LA_2020 = addNegativeSamples(kaggle_LA_2020)
print(kaggle_LA_2020.shape)

(49527, 49)


In [6]:
# average the position of start point and end point of accident to represent the position of the overall accident
print(kaggle_LA_2020.head())
kaggle_LA_2020["Lat"] = (kaggle_LA_2020["Start_Lat"]+kaggle_LA_2020["End_Lat"])/2
kaggle_LA_2020["Lng"] = (kaggle_LA_2020["Start_Lng"]+kaggle_LA_2020["End_Lng"])/2
# extract needed features and label
kaggle_LA_2020_new = kaggle_LA_2020[["Lat", "Lng", "Crossing", "Junction", "Railway", "Station", "Accident"]]
print(kaggle_LA_2020_new.shape)

          ID  Severity          Start_Time             End_Time  Start_Lat  \
0  A-1735388         2 2020-11-28 01:33:00  2020-11-28 03:13:16  33.963490   
1  A-1735405         2 2020-11-10 02:32:00  2020-11-10 04:46:20  34.152964   
2  A-1735564         2 2020-11-20 08:47:00  2020-11-20 12:17:14  34.051600   
3  A-1735606         2 2020-12-22 12:38:30  2020-12-22 20:46:30  33.981910   
4  A-1735607         2 2020-12-10 20:34:30  2020-12-10 21:33:30  34.135160   

    Start_Lng    End_Lat     End_Lng  Distance(mi)  \
0 -118.370694  33.963061 -118.370409         0.034   
1 -118.278776  34.153274 -118.277476         0.077   
2 -118.458568  34.052739 -118.458171         0.082   
3 -118.288154  33.982370 -118.281096         0.406   
4 -118.358750  34.139809 -118.365350         0.496   

                                         Description  ...   Stop  \
0  Incident on S LA CIENEGA BLVD near W FLORENCE ...  ...   True   
1                            EB 134 JWO I5. 3 VEH TC  ...  False   
2 

In [7]:
# sort the nodes of graph based on their id value
nodes_list = sorted(list(G.nodes))
tmp = nodes_list[100]
print(G.nodes[tmp])

{'y': 34.1235604, 'x': -118.226157, 'highway': 'stop', 'street_count': 5}


In [8]:
temp = 0
for edge in G.edges:
    print(G[edge[0]][edge[1]][0])
    if temp>6:
        break
    temp+=1

{'osmid': [907145138, 895315641, 895315642, 398770659], 'lanes': '6', 'name': 'National Boulevard', 'highway': 'secondary', 'maxspeed': '35 mph', 'oneway': False, 'length': 88.746, 'geometry': <shapely.geometry.linestring.LineString object at 0x7fa64b4f6b50>}
{'osmid': [398770658, 759468526, 759468527], 'oneway': False, 'lanes': '4', 'name': 'National Boulevard', 'highway': 'secondary', 'maxspeed': '35 mph', 'length': 102.25999999999999, 'tunnel': 'yes', 'geometry': <shapely.geometry.linestring.LineString object at 0x7fa64b4f69d0>}
{'osmid': 398771138, 'lanes': '5', 'name': 'National Boulevard', 'highway': 'secondary', 'maxspeed': '35 mph', 'oneway': False, 'length': 19.948999999999998, 'geometry': <shapely.geometry.linestring.LineString object at 0x7fa64b4f6510>}
{'osmid': [404964730, 398771139], 'lanes': ['5', '4'], 'name': 'National Boulevard', 'highway': 'secondary', 'maxspeed': '35 mph', 'oneway': False, 'length': 109.153, 'geometry': <shapely.geometry.linestring.LineString object

In [9]:
# link each node of graph with the nearest position in the accident dataset, then introduce the features of accident point as the features of node
nodes_num = len(nodes_list)
print("nodes_num", nodes_num)
feature_matrix = np.zeros((nodes_num,7))
labels = []
times = 0
num_1s = 0
num_0s = 0
for node in nodes_list:
    lats = G.nodes[node]["y"]
    lngs = G.nodes[node]["x"]
    if "street_count" in G.nodes[node]:
        street_count = G.nodes[node]["street_count"]
    else:
        street_count = 0
    dist_min = float("inf")
    lat_accident = np.array(kaggle_LA_2020_new["Lat"])
    lng_accident = np.array(kaggle_LA_2020_new["Lng"])
    dist = haversine(lngs, lats, lng_accident, lat_accident)
    dist_index = np.argmin(dist)
    dist_tmp = np.min(dist)
    if dist_tmp<dist_min:
        dist_min = dist_tmp
        crossing = int(kaggle_LA_2020_new["Crossing"].iloc[dist_index])
        junction = int(kaggle_LA_2020_new["Junction"].iloc[dist_index])
        railway = int(kaggle_LA_2020_new["Railway"].iloc[dist_index])
        station = int(kaggle_LA_2020_new["Station"].iloc[dist_index])
    feature_matrix[times, 0] = lats
    feature_matrix[times, 1] = lngs
    feature_matrix[times, 2] = street_count
    feature_matrix[times, 3] = crossing
    feature_matrix[times, 4] = junction
    feature_matrix[times, 5] = railway
    feature_matrix[times, 6] = station
    labels.append(kaggle_LA_2020_new["Accident"].iloc[dist_index])
    times+=1
    if kaggle_LA_2020["Accident"].iloc[dist_index]==1:
        num_1s+=1
    else:
        num_0s+=1
print(len(labels))
print("number of 1s:", num_1s)
print("number of 0s:", num_0s)
        

nodes_num 49248
49248
number of 1s: 16359
number of 0s: 32889


In [11]:
"""
get the minimal value and maximal value of latitude and longitude to cover the whole LA. After that, split this rectangle area into 400
blocks and record the intreval in the direction of latitude and longtiude. Based on the interval, calculate the node to find which block
it belongs to, then record the coordinate (xx, yy) of block
"""
lat_min = np.min(feature_matrix[:, 0])
lat_max = np.max(feature_matrix[:, 0])
lng_min = np.min(feature_matrix[:, 1])
lng_max = np.max(feature_matrix[:, 1])
# 20 x 20 = 400 blocks
lat_interval = (lat_max-lat_min)/20
lng_interval = (lng_max-lng_min)/20
save_dict = {}
for i in range(feature_matrix.shape[0]):
    xx = min(int((feature_matrix[i, 0]-lat_min)/lat_interval), 19)
    yy = min(int((feature_matrix[i, 1]-lng_min)/lng_interval), 19)
    if tuple([xx, yy]) not in save_dict:
        save_dict[tuple([xx, yy])] = [i]
    else:
        save_dict[tuple([xx, yy])].append(i)

In [14]:
"""
Traverse the dictionary, build data for each small graph inside, including build node features, labels, edge indexes and edge attributes
"""

import torch
from torch_geometric.data import Data

graphs = []
for key in save_dict:
    indexes = save_dict[key]
    if len(indexes)<=5:
        continue
    node_features = []
    nodes = []
    node_labels = []
    for index in indexes:
        node_features.append(list(feature_matrix[index, :]))
        node_labels.append(labels[index])
        nodes.append(nodes_list[index])
    node_features = np.array(node_features)
    x = torch.tensor(node_features, dtype=torch.float)
    node_labels = np.array(node_labels)
    y = torch.tensor(node_labels)
    nodes_dict1 =  dict()
    val = 0
    for item in nodes:
        nodes_dict1[item] = val
        val+=1
    
    sub_G = G.subgraph(nodes)
    sources = []
    targets = []
    edge_attributes = []  # length, bridge, lanes, oneway, maxspeed, tunnel
    for edge in sub_G.edges:
        sources.append(nodes_dict1[edge[0]])
        targets.append(nodes_dict1[edge[1]])
        edge_attr = sub_G[edge[0]][edge[1]][0]
        if "length" in edge_attr:
            length = edge_attr["length"]
        else:
            length = 0
        
        if "bridge" in edge_attr:
            if edge_attr["bridge"]=="yes":
                bridge=1
            else:
                bridge=0
        else:
            bridge=0
            
        if "tunnel" in edge_attr:
            if edge_attr["tunnel"]=="yes":
                tunnel=1
            else:
                tunnel=0
        else:
            tunnel=0
        
        if "lanes" in edge_attr:
            if type(edge_attr["lanes"]) == list:
                temp = list(map(int, edge_attr["lanes"]))
                lanes = max(temp)
            else:
                lanes = int(edge_attr["lanes"])
        else:
            lanes = 0
        
        if "oneway" in edge_attr:
            oneway = int(edge_attr["oneway"])
        else:
            oneway = 0
            
        if "maxspeed" in edge_attr:
            if type(edge_attr["maxspeed"]) == list:
                temp = list(map(lambda x: int(x.split()[0]), edge_attr["maxspeed"]))
                maxspeed = max(temp)
            else:
                maxspeed = int(edge_attr["maxspeed"].split()[0])
        else:
            maxspeed = 0
        edge_attributes.append([length, bridge, tunnel, lanes, oneway, maxspeed])
    edge_attributes = np.array(edge_attributes)
    edge_attr = torch.tensor(edge_attributes, dtype=torch.float)
    edge_index = torch.tensor([sources, targets], dtype=torch.long)
    data = Data(edge_index=edge_index, x=x, y=y, edge_attr=edge_attr)
    graphs.append(data)
print("total number:", len(graphs))

tensor([[106, 106, 106, 107, 107, 107, 108, 108, 108, 236],
        [247, 206, 212, 170, 108, 247, 306, 107, 171,   8]])
tensor([[ 89,  89, 224, 224, 224, 225, 225, 225, 225, 226],
        [ 88, 187,  47,  76, 304, 226,  29,  77, 406, 227]])
tensor([[218, 218, 218,  89,  89,  89,   9,   9,   9, 350],
        [ 78, 141, 134, 116, 358,  47,  21, 171, 314, 227]])
tensor([[281, 281, 281, 282, 282, 282, 125, 125, 125, 126],
        [ 29,  47, 133,  40, 161,  39, 182, 126,  81, 205]])
tensor([[168, 169, 169, 169, 169, 170, 170, 170, 113, 113],
        [169, 170, 168, 239,  38, 169, 193,  39, 114, 203]])
tensor([[384, 385, 385, 385,  13,  13,  13,  13,  74,  74],
        [391, 392, 189, 391,  29, 357,  42,   4,  75, 125]])
tensor([[ 14,  14,  55,  22,  22,  56,  56,  23,  57,  57],
        [ 55,  23,  30,  74,  75,  89,  69,  24, 100,  27]])
tensor([[  5,   5,   5, 304, 304, 304, 305, 305, 305, 201],
        [  1,   0,  47, 305,  76,  88, 304, 339, 337, 202]])
tensor([[165, 165, 165, 165,  59

In [ ]:
# Model
from torch_geometric.nn import GraphConv, GCNConv, SAGEConv, GATConv
import torch
import torch.nn as nn
from torch.nn import Linear
import torch.nn.functional as F

class GNNModel(nn.Module):
    def __init__(self, classes):
        super(GNNModel, self).__init__()
        self.conv1 = GATConv(7, 64)
        self.conv2 = GATConv(64, 64)
        self.conv3 = GATConv(64, 64)
        self.linear = Linear(64, classes)
    
    def forward(self, data):
        x, edge_index, edge_attr = data.x, data.edge_index, data.edge_attr
        x = self.conv1(x, edge_index, edge_attr)
        x = F.relu(x)
        x = F.dropout(x, 0.4)
        x = self.conv2(x, edge_index, edge_attr)
        x = F.relu(x)
        x = F.dropout(x, 0.4)
        x = self.conv3(x, edge_index, edge_attr)
        x = F.relu(x)
        x = F.dropout(x, 0.4)
        x = self.linear(x)
        return x

In [ ]:
# overall training pipeline
from torch_geometric.data import DataLoader
from torch.optim import Adam, AdamW

batch_size = 8
train_sets = graphs[:-10]
test_sets = graphs[-10:]
train_loader = DataLoader(train_sets, batch_size=batch_size)
model = GNNModel(2)
optimizer = AdamW(model.parameters(), lr=1e-4, weight_decay=5e-5)
criterion = nn.CrossEntropyLoss()
best_loss = float("inf")
epochs = 200
interval = 50

for epoch in range(epochs):
    model.train()
    losses = 0
    nums = len(train_sets)
    for _, data in enumerate(train_loader):
        data = data
        labels = data.y
        predicts = model(data)
        loss = criterion(predicts, labels)
        losses+=loss.item()
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    print(f"epoch:{epoch}, loss:{losses/nums}")
    avg_loss = losses/nums
    if avg_loss<best_loss:
        best_loss = avg_loss
    if epoch%interval == 0:
        model.eval()
        acc_avg = 0
        times = 0
        for _, data in enumerate(test_sets):
            _, pred = model(data).max(dim=1)
            correct = int(pred.eq(data.y).sum().item())
            acc = correct / int(data.x.shape[0])
            acc_avg+=acc
            times+=1
        print(f"acc: {acc_avg/times}")